## Criando o primeiro dataframe com dados, com os campos (Funcionario,Salario)

In [0]:
%python
empresa1 = spark.createDataFrame(
[
("Joao Santos",2000),
("Carlos Fernandez",3400)
],['Funcionario', 'Salario'] )
display(empresa1)
empresa1.printSchema()

Funcionario,Salario
Joao Santos,2000
Carlos Fernandez,3400


root
 |-- Funcionario: string (nullable = true)
 |-- Salario: long (nullable = true)



## Setado caminho, armazenando em um atributo

In [0]:
%python
parquetpath = "dbfs:/FileStore/tables/delta/schema_evolution/parquet"

## Criando arquivos parquet com base no primeiro dataframe

In [0]:
%python
(
empresa1
.write
.format("parquet")
.save("/FileStore/tables/delta/schema_evolution/parquet")
)
spark.read.parquet(parquetpath).show()

+----------------+-------+
|     Funcionario|Salario|
+----------------+-------+
|Carlos Fernandez|   3400|
|     Joao Santos|   2000|
+----------------+-------+



## Criando o segundo dataframe com dados, acrescentando novos campos (Setor,Comissao)

In [0]:
%python
empresa2 = spark.createDataFrame(
[
("Financeiro",240),
("Marketing",540)
],['Setor', 'Comissao'] )
display(empresa2)
empresa2.printSchema()

Setor,Comissao
Financeiro,240
Marketing,540


root
 |-- Setor: string (nullable = true)
 |-- Comissao: long (nullable = true)



## Apesar de colocar "append"no parquet, não houve evolução do esquema, as colunas foram substituídas

In [0]:
%python
empresa2.write.mode("append").parquet(parquetpath)
spark.read.parquet(parquetpath).show()

+----------+--------+
|     Setor|Comissao|
+----------+--------+
|      null|    null|
|      null|    null|
|Financeiro|     240|
| Marketing|     540|
+----------+--------+



##Vamos gerar o Schema Evolution com as tabelas Delta

In [0]:
%python
deltapath = "/FileStore/tables/delta/schema_evolution/delta"
(
empresa1
.write
.format("delta")
.save("/FileStore/tables/delta/schema_evolution/delta")
)
spark.read.format("delta").load(deltapath).show()

+----------------+-------+
|     Funcionario|Salario|
+----------------+-------+
|Carlos Fernandez|   3400|
|     Joao Santos|   2000|
+----------------+-------+



##Vamos realizar um "merge" entre os dataframes, note que agora conseguirá realizar a junção entre os schemas. Os dados inexistentes foram acrescidos de nulos

In [0]:
%python
(
empresa2
.write
.format("delta")
.mode("append")
.option("mergeSchema", "true")
.save(deltapath)
)
spark.read.format("delta").load(deltapath).show()

+----------------+-------+----------+--------+
|     Funcionario|Salario|     Setor|Comissao|
+----------------+-------+----------+--------+
|Carlos Fernandez|   3400|      null|    null|
|     Joao Santos|   2000|      null|    null|
|            null|   null|Financeiro|     240|
|            null|   null| Marketing|     540|
+----------------+-------+----------+--------+



##Vamos inserir novos dados e aplicar o "append" para verificar como funcionará a inserção

In [0]:
%python
empresa3 = spark.createDataFrame(
[
("Sandra Lemos",672),
("Carla Soares",966),
],
['Funcionario', 'Comissao']
)

## Vamos acrescentar dados mais dados e verificar a inclusão apenas de alguns campos

In [0]:
%python
(
empresa3
.write
.format("delta")
.mode("append")
.option("mergeSchema", "true")
.save(deltapath)
)
spark.read.format("delta").load(deltapath).show()

+----------------+-------+----------+--------+
|     Funcionario|Salario|     Setor|Comissao|
+----------------+-------+----------+--------+
|Carlos Fernandez|   3400|      null|    null|
|    Sandra Lemos|   null|      null|     672|
|    Carla Soares|   null|      null|     966|
|     Joao Santos|   2000|      null|    null|
|            null|   null|Financeiro|     240|
|            null|   null| Marketing|     540|
+----------------+-------+----------+--------+



##Vamos fazer sobregravação dos formatos delta, variando os modos , quando o merge está ativo "option=mergeSchema, mode=overwrite"

In [0]:
%python
(
empresa3
.write
.format("delta")
.mode("overwrite")
.option("mergeSchema", "true")
.save(deltapath)
)
spark.read.format("delta").load(deltapath).show()

+------------+-------+-----+--------+
| Funcionario|Salario|Setor|Comissao|
+------------+-------+-----+--------+
|Sandra Lemos|   null| null|     672|
|Carla Soares|   null| null|     966|
+------------+-------+-----+--------+



## Vamos sobrescrever toda a tabela, perceba as mudancas dos campos que ficaram e dos registros. "option=overwriteschema , mode=overwrite"

In [0]:
%python
(
empresa3
.write
.format("delta")
.option("overwriteSchema", "true")
.mode("overwrite")
.save(deltapath)
)
spark.read.format("delta").load(deltapath).show()

+------------+--------+
| Funcionario|Comissao|
+------------+--------+
|Sandra Lemos|     672|
|Carla Soares|     966|
+------------+--------+



## Vamos criar uma tabela delta com referência aos parquet (delta) criados

In [0]:
%sql
CREATE TABLE tab_empresa(
Funcionario STRING,
Comissao long
)
USING delta
LOCATION "/FileStore/tables/delta/schema_evolution/delta"

## Vamos listar o histórico gerado de todas as nossas mudanças

In [0]:
%sql
DESCRIBE HISTORY '/FileStore/tables/delta/schema_evolution/delta'

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
4,2023-07-27T00:34:37.000+0000,3749819615057738,andersonfaro@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1243705193155246),0726-224338-nu3hhmdt,3,WriteSerializable,false,"Map(numFiles -> 2, numOutputRows -> 2, numOutputBytes -> 1858)",null,Databricks-Runtime/10.4.x-scala2.12
3,2023-07-27T00:33:49.000+0000,3749819615057738,andersonfaro@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1243705193155246),0726-224338-nu3hhmdt,2,WriteSerializable,false,"Map(numFiles -> 2, numOutputRows -> 2, numOutputBytes -> 1858)",null,Databricks-Runtime/10.4.x-scala2.12
2,2023-07-27T00:32:40.000+0000,3749819615057738,andersonfaro@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(1243705193155246),0726-224338-nu3hhmdt,1,WriteSerializable,true,"Map(numFiles -> 2, numOutputRows -> 2, numOutputBytes -> 1858)",null,Databricks-Runtime/10.4.x-scala2.12
1,2023-07-27T00:31:24.000+0000,3749819615057738,andersonfaro@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(1243705193155246),0726-224338-nu3hhmdt,0,WriteSerializable,true,"Map(numFiles -> 2, numOutputRows -> 2, numOutputBytes -> 1786)",null,Databricks-Runtime/10.4.x-scala2.12
0,2023-07-27T00:29:38.000+0000,3749819615057738,andersonfaro@gmail.com,WRITE,"Map(mode -> ErrorIfExists, partitionBy -> [])",null,List(1243705193155246),0726-224338-nu3hhmdt,null,WriteSerializable,true,"Map(numFiles -> 2, numOutputRows -> 2, numOutputBytes -> 1873)",null,Databricks-Runtime/10.4.x-scala2.12


## Listando todas as versões que podemos utilizar

In [0]:
%sql
SELECT * FROM delta. `/FileStore/tables/delta/schema_evolution/delta`
VERSION AS OF 2

Funcionario,Salario,Setor,Comissao
Carlos Fernandez,3400,null,null
Sandra Lemos,null,null,672
Carla Soares,null,null,966
Joao Santos,2000,null,null
null,null,Financeiro,240
null,null,Marketing,540
